In [1]:
%%capture
!pip install gdown

In [2]:
%%capture
import gdown
URL = 'https://drive.google.com/file/d/1GBBkwDzJ7HI1b-i_d9kpn-VWpaiHxRUj/view?usp=sharing'
gdown.download(URL, 'gnss_log.txt', quiet=True, fuzzy=True)

In [15]:
import pandas as pd

column_names = [
    "Status",
    "UnixTimeMillis",
    "SignalCount",
    "SignalIndex",
    "ConstellationType",
    "Svid",
    "CarrierFrequencyHz",
    "Cn0DbHz",
    "AzimuthDegrees",
    "ElevationDegrees",
    "UsedInFix",
    "HasAlmanacData",
    "HasEphemerisData",
    "BasebandCn0DbHz"
]

df = pd.read_csv(
    'gnss_log.txt',
    comment="#",
    names=column_names,
    on_bad_lines='skip',
)
df.head(3)

,Status,UnixTimeMillis,SignalCount,SignalIndex,ConstellationType,Svid,CarrierFrequencyHz,Cn0DbHz,AzimuthDegrees,ElevationDegrees,UsedInFix,HasAlmanacData,HasEphemerisData,BasebandCn0DbHz
0,Status,NaN,5,0,5,40,1561097980,32.64,355.0,65.0,0,1,1,28.642162
1,Status,NaN,5,1,1,7,1575420030,19.70,331.0,61.0,0,1,1,15.699816
2,Status,NaN,5,2,3,24,1603124990,20.15,194.0,27.0,0,1,1,16.147501


In [16]:
df['Svid'].nunique()

36

In [24]:
def get_constellation_name(constellation_type):
    constellation_map = {
        1: "GPS",
        3: "Glonass",
        4: "QZSS",
        5: "BeiDou",
        6: "Galileo",
    }
    return constellation_map.get(constellation_type, "UNKNOWN")

df["satellite_type"] = df["ConstellationType"].apply(get_constellation_name)
df.head(3)

,Status,UnixTimeMillis,SignalCount,SignalIndex,ConstellationType,Svid,CarrierFrequencyHz,Cn0DbHz,AzimuthDegrees,ElevationDegrees,UsedInFix,HasAlmanacData,HasEphemerisData,BasebandCn0DbHz,norad_cat_id,satellite_altitude,satellite_azimuth,altitude_delta,azimuth_delta,satellite_type
0,Status,NaN,5,0,5,40,1561097980,32.64,355.0,65.0,0,1,1,28.642162,37256,63.559476,344.626543,1.440524,10.373457,BeiDou
1,Status,NaN,5,1,1,7,1575420030,19.70,331.0,61.0,0,1,1,15.699816,37256,63.559476,344.626543,-2.559476,-13.626543,GPS
2,Status,NaN,5,2,3,24,1603124990,20.15,194.0,27.0,0,1,1,16.147501,43564,22.118412,191.694073,4.881588,2.305927,Glonass


In [25]:
df['BasebandCn0DbHz'].describe()

,BasebandCn0DbHz
count,215352.000000
mean,20.634785
std,7.172838
min,7.000929
25%,14.656185
50%,21.056905
75%,25.775250
max,41.098965


In [35]:
df = df[df['BasebandCn0DbHz'] > 25]
df['Svid'].nunique()

34

In [28]:
min_unix_time = df['UnixTimeMillis'].min(skipna=True)
max_unix_time = df['UnixTimeMillis'].max(skipna=True)

min_datetime_utc = pd.to_datetime(min_unix_time, unit='ms', utc=True)
max_datetime_utc = pd.to_datetime(max_unix_time, unit='ms', utc=True)

min_datetime_local = min_datetime_utc.tz_convert('Asia/Ho_Chi_Minh')
max_datetime_local = max_datetime_utc.tz_convert('Asia/Ho_Chi_Minh')

print("Start:", min_datetime_local)
print("End:", max_datetime_local)

Start: 2025-03-01 11:04:18.606000+07:00
End: 2025-03-01 12:57:48+07:00


In [29]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/13R-tVolpJrSBPHcqLKms-9dGeL2nHMm3/view?usp=share_link'
gdown.download(URL, 'satellites.csv', quiet=True, fuzzy=True)

In [30]:
import pandas as pd

satellites_df = pd.read_csv('satellites.csv')
satellites_df.head(3)

,time,satellite_id,altitude,azimuth,satellite_type
0,2025-03-01T12:01:03.000Z,42738,48.758917,61.534026,QZSS
1,2025-03-01T12:01:03.000Z,42917,55.661479,132.829331,QZSS
2,2025-03-01T12:01:03.000Z,42965,52.446159,85.069715,QZSS


In [38]:
from scipy.spatial import cKDTree

constellation_satellites = satellites_df[satellites_df['satellite_type'] == 'GPS']
observed_satellites = df[df['satellite_type'] == 'GPS']

tree = cKDTree(constellation_satellites[['altitude', 'azimuth']].values)
distances, indices = tree.query(observed_satellites[['ElevationDegrees', 'AzimuthDegrees']].values)

observed_satellites['norad_cat_id'] = constellation_satellites.iloc[indices]['satellite_id'].values
observed_satellites['satellite_altitude'] = constellation_satellites.iloc[indices]['altitude'].values
observed_satellites['satellite_azimuth'] = constellation_satellites.iloc[indices]['azimuth'].values

observed_satellites['altitude_delta'] = observed_satellites['ElevationDegrees'] - observed_satellites['satellite_altitude']
observed_satellites['azimuth_delta'] = observed_satellites['AzimuthDegrees'] - observed_satellites['satellite_azimuth']

observed_satellites.head()

# df.drop_duplicates(subset=['constellation', 'svid'])[['constellation', 'svid', 'norad_cat_id']].to_csv('svid_to_norad_id.csv', index=False)

<ipython-input-38-26b5a0531e6e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observed_satellites['norad_cat_id'] = constellation_satellites.iloc[indices]['satellite_id'].values
<ipython-input-38-26b5a0531e6e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observed_satellites['satellite_altitude'] = constellation_satellites.iloc[indices]['altitude'].values
<ipython-input-38-26b5a0531e6e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,Status,UnixTimeMillis,SignalCount,SignalIndex,ConstellationType,Svid,CarrierFrequencyHz,Cn0DbHz,AzimuthDegrees,ElevationDegrees,UsedInFix,HasAlmanacData,HasEphemerisData,BasebandCn0DbHz,norad_cat_id,satellite_altitude,satellite_azimuth,altitude_delta,azimuth_delta,satellite_type
1771,Status,NaN,8,4,1,1,1575420030,29.96,146.0,45.0,0,1,1,25.956028,26690,9.132105,144.000521,35.867895,1.999479,GPS
1779,Status,NaN,8,4,1,1,1575420030,32.38,146.0,45.0,0,1,1,28.381214,26690,9.132105,144.000521,35.867895,1.999479,GPS
1787,Status,NaN,8,4,1,1,1575420030,32.16,146.0,45.0,0,1,1,28.155577,26690,9.132105,144.000521,35.867895,1.999479,GPS
1795,Status,NaN,8,4,1,1,1575420030,31.64,146.0,45.0,0,1,1,27.640657,26690,9.132105,144.000521,35.867895,1.999479,GPS
1803,Status,NaN,8,4,1,1,1575420030,31.00,146.0,45.0,0,1,1,27.000832,26690,9.132105,144.000521,35.867895,1.999479,GPS


In [40]:
observed_satellites['norad_cat_id'].nunique()

13

In [39]:
observed_satellites['Svid'].nunique()

13

In [42]:
observed_satellites.groupby(['norad_cat_id', 'Svid']).agg(
    count=('BasebandCn0DbHz', 'size'),  # Count of rows for each unique combination
    max_cn0=('BasebandCn0DbHz', 'max')   # Maximum BasebandCn0DbHz for each group
)

count    max_cn0
norad_cat_id Svid                  
26407        14      173  36.289845
             22       66  30.492933
26690        1        14  28.381214
             3       196  37.528244
28129        3        15  26.104582
28190        17      383  37.487040
             19      355  40.231880
28474        1       878  36.752174
             2      1452  36.794580
28874        17     2758  38.223900
             19      394  38.698370
32711        7      1102  32.408607
             30      268  34.776540
39533        7       674  30.318218
             14      169  31.730171
             17      249  35.587067
             30     1899  38.704150
40105        4       139  33.956840
             7       928  36.906610
             9       689  39.354828
40730        2       371  36.769375
             8      1206  40.185883
             27        3  25.887184
43873        4        55  31.822138
46826        14      751  39.600082
             22      519  41.098965
             30      357  36.045082
62339        1      1898  37.561657
             2       888  34.589542